In [1]:
import convokit

In [2]:
from convokit import Forecaster, Corpus, download

In [3]:
MAX_LENGTH = 80

In [5]:
from convokit.forecaster.CRAFTModel import CRAFTModel

In [6]:
craft_model = CRAFTModel(device_type="cpu", options={'validation_size': 0.2,
                                                              'train_epochs': 5
                                                             })

Initializing CRAFT model with options:
{'validation_size': 0.2, 'train_epochs': 5, 'hidden_size': 500, 'encoder_n_layers': 2, 'context_encoder_n_layers': 2, 'decoder_n_layers': 2, 'dropout': 0.1, 'batch_size': 64, 'clip': 50.0, 'learning_rate': 1e-05, 'print_every': 10, 'max_length': 80, 'trained_model_output_filepath': 'finetuned_model.tar'}
Loading saved parameters...
Building encoders, decoder, and classifier...
Models built and ready to go!


In [7]:
forecaster = Forecaster(forecaster_model = craft_model,
                        forecast_mode = 'past',
                        convo_structure="linear",
                        text_func = lambda utt: utt.meta["tokens"][:(MAX_LENGTH-1)],
                        label_func = lambda utt: int(utt.meta['comment_has_personal_attack']),
                        forecast_feat_name="prediction", forecast_prob_feat_name="pred_score",
                        use_last_only = True,
                        skip_broken_convos=False
                       )

In [8]:
corpus = Corpus(filename=download("conversations-gone-awry-corpus"))

Dataset already exists at /Users/calebchiam/.convokit/downloads/conversations-gone-awry-corpus


In [10]:
from convokit.forecaster.CRAFT import craft_tokenize

In [11]:
for utt in corpus.iter_utterances():
    utt.add_meta("tokens", craft_tokenize(craft_model.voc, utt.text))

In [ ]:
forecaster.fit(corpus, selector = lambda convo: convo.meta["split"] == "train",
               ignore_utterances = lambda utt: utt.meta["is_section_header"])

Building optimizers...
Starting Training!
Will train for 155 iterations
Initializing ...
Training...
Iteration: 10; Percent complete: 6.5%; Average loss: 0.4000


In [ ]:
forecaster.forecast_mode = "past"

In [ ]:
forecaster.transform(corpus)